In [3]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import itertools
import re 
import time


In [9]:
#Instancia Browser

chrome = webdriver.Chrome("chromedriver.exe")

chrome.get('https://www.quintoandar.com.br/alugar/imovel/belo-horizonte-mg-brasil/apartamento/3-quartos/de-500-a-2500-aluguel/1-vagas')
time.sleep(0.5)

body = chrome.find_element_by_class_name("class.s5ca2ou-0.iHysKY")

no_pageDowns = 50

#Scroll Down  while no_pagedows > 0.

while no_pageDowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    no_pageDowns -= 1


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"s5ca2ou-0.iHysKY"}
  (Session info: chrome=75.0.3770.100)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)


In [3]:
#Cria Headers e envia HTTP Request para o site - conexão com o site 

browser = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}) # Mimic a browser

site =  'https://www.quintoandar.com.br/alugar/imovel/belo-horizonte-mg-brasil/apartamento/3-quartos/de-500-a-2500-aluguel/1-vagas'

response = requests.get(site, headers = browser)

response.status_code

200